# 畳み込みニューラルネットワーク

In [2]:
# 基本的なCNNのインスタンス化
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Using TensorFlow backend.


重要なのはCNNの入力テンソルが(image_hight, image_width, image_channels)で、バッチ入力を含まない形状であるということ。

In [3]:
# CNNのアーキテクチャ
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


各Conv2D層とMaxPooling2D層の出力は三次元テンソルであり、形状は(hight, width, channels)となっている。
hight, widthはネットワークが深くなるに連れて縮小傾向にある。 

In [4]:
# CNNの上に分類器を追加
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

最後のテンソル出力を全結合分類器に供給する。最初に３次元の出力を１次元に平坦化してから、Dense層に追加する。

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

> flatten_1 (Flatten)          (None, 576)               0         

ここでベクトルが平坦化されていることがわかる。

In [6]:
# MNISTの画像でCNNを訓練
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 18s 305us/step - loss: 0.1827 - acc: 0.94336s - lo - ET - ETA: 1s - loss
Epoch 2/5
60000/60000 [==============================] - 15s 257us/step - loss: 0.0484 - acc: 0.9854
Epoch 3/5
60000/60000 [==============================] - 15s 256us/step - loss: 0.0331 - acc: 0.9897
Epoch 4/5
60000/60000 [==============================] - 15s 257us/step - loss: 0.0235 - acc: 0.99236s - loss 
Epoch 5/5
60000/60000 [==============================] - 15s 256us/step - loss: 0.0188 - acc: 0.9940


In [7]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 1s 125us/step


0.9905